Промаркируем топиками все единицы векторной базы

Инсталлы

In [1]:
! pip install --upgrade --quiet yandex-cloud-ml-sdk


Импорты

In [40]:
import os
from yandex_cloud_ml_sdk import YCloudML
import json
import pandas as pd
import ast


Функция для запросов к модели

In [62]:


folder_id = os.environ["FOLDER_ID"]
api_key = os.environ["API_KEY"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)
model = sdk.models.completions("yandexgpt", model_version="rc")


valid_topics = 'проходной балл, меганаправления, количество бюджетных мест, цена года обучения, общежития, индивидуальные достижения, компании-партнеры, стажировки, математика, программирование, кафедры, преподаватели, стипендии, корпуса, ремонт, айти-этаж, лаборатории, подача заявления, дни открытых дверей, чат приемки, чаты меганаправлений, языки программирования, баллы на общежитие, сертификаты на общежитие, бви, магистратура, обучение, программа, административные вопросы, профбюро, мероприятия, СпецВО, БВО, реклама'.split(', ')

def tag_query(query):
    with open('prompts/tagging_prompt.md', 'r', encoding='utf-8') as f:
        system_prompt = f.read()
    return model.run(system_prompt + "\n\n Входной запрос:\n" + query).text
tag_query('''Это не сон! Альфа-Банк дарит 500 000 рублей студентам за их идеи 🚀

Вот как можно выиграть приз — участвуйте во всероссийском хакатоне Alfa Hack и создавайте свой AutoML!

Команда Лаборатории машинного обучения Альфа-Банка подготовила кейсы для студентов, которые хотят прокачать свои навыки и показать свои возможности на всю 
страну 🦾

За несколько недель вы сможете:
💡 Решить реальные бизнес-задачи Альфа-Банка и получить практический опыт
💡Пообщаться и завязать контакты с ведущими экспертами отрасли
💡 Выиграть денежный приз и получить фаст-трек в команду Альфы
▶ Кто может участвовать?

Студенты Data Science, Data Analysis, ML-Engineering направлений и других направлений.

Первый этап пройдет онлайн, а финалисты встретятся в Москве на очном финале 🔥

Собирайте команду и регистрируйтесь до 31 октября по ссылке: https://alfa.me/0D7ck4 

Если команды пока нет — не волнуйтесь, мы поможем найти единомышленников 🤝

Скорее подавайте заявку, будем создавать технологии будущего вместе ❤''')

'реклама\n'

Обработка диалогов

In [63]:
df_threads = pd.read_csv('vk_chunks_df.csv')
for query in list(df_threads['dialog_plain']):
    ans = tag_query(query)

    topics = ans.strip().replace('[', '').replace(']', '').replace('\n\n\u200b', '').split(', ')
    res = []
    for i in topics:
        if i  in valid_topics:
            res.append(i)
    topics = res


    with open('../data/tags/vk_tags.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    data[query] = topics
    with open('../data/tags/vk_tags.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

Обработка  постов